In [1]:
import pandas as pd
from cobra import io
from cobra.core import Metabolite, Reaction
import pythoncyc
from Functions import *

import logging
logging.basicConfig()
logger = logging.getLogger('logger')


fname = "Data/Soy_core_model_GA.sbml"
model = io.sbml.read_sbml_model(fname,use_fbc_package=True)

## Generate leaf model

In [2]:
leaf_model = model.copy()



Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [3]:
df_matureleaf_content_1 = pd.DataFrame(data={"Series":[1,2,3,4],
                                           "starch_dawn (g/100gFW)":[0.4245676464308603,0.4974291547005305,
                                                                     0.8895379007845832,1.2526663271935523],
                                           "hemicellulose_dawn (g/100gFW)":[1.089463383389083,
                                                                            1.8922736237500428,
                                                                            0.8616451473404596,
                                                                            1.1221330617548415],
                                           "simple sugars_dawn (g/100gFW)":[0.2719414988257076,
                                                                            0.6501251398452184,
                                                                            0.5686189976083096,
                                                                            0.5533809766290336],
                                           "sucrose_dawn (g/100gFW)":[0.2719414988257076,0.454127308166961,
                                                                      0.09770538968660913,0],
                                           "starch_dusk (g/100gFW)":[1.3284677247171013,1.600011533626287,
                                                                     2.6254282537133005,1.0978029358571406],
                                           "hemicellulose_dusk (g/100gFW)":[1.089463383389083,
                                                                            0.6633103814170216,
                                                                            0.9510687588805857,
                                                                            1.0721624730031074],
                                           "simple sugars_dusk (g/100gFW)":[0.03202263184115184,
                                                                            0.35337416237039254,
                                                                            0.4138784209550903,
                                                                            0.3891937806184267],
                                           "sucrose_dusk (g/100gFW)":[0.46007757454985465,0.18700390989930638,
                                                                      0.3964371762172485,0.14206905936682546]})
df_matureleaf_content_1.set_index("Series",drop=True,inplace=True)
df_matureleaf_content_1["Δstarch"] = df_matureleaf_content_1["starch_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["starch_dawn (g/100gFW)"]
df_matureleaf_content_1["Δhemicellulose"] = df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsimple sugars"] = df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsucrose"] = df_matureleaf_content_1["sucrose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["sucrose_dawn (g/100gFW)"]
df_matureleaf_content_1

,starch_dawn (g/100gFW),hemicellulose_dawn (g/100gFW),simple sugars_dawn (g/100gFW),sucrose_dawn (g/100gFW),starch_dusk (g/100gFW),hemicellulose_dusk (g/100gFW),simple sugars_dusk (g/100gFW),sucrose_dusk (g/100gFW),Δstarch,Δhemicellulose,Δsimple sugars,Δsucrose
Series,,,,,,,,,,,,
1,0.424568,1.089463,0.271941,0.271941,1.328468,1.089463,0.032023,0.460078,0.903900,0.000000,-0.239919,0.188136
2,0.497429,1.892274,0.650125,0.454127,1.600012,0.663310,0.353374,0.187004,1.102582,-1.228963,-0.296751,-0.267123
3,0.889538,0.861645,0.568619,0.097705,2.625428,0.951069,0.413878,0.396437,1.735890,0.089424,-0.154741,0.298732
4,1.252666,1.122133,0.553381,0.000000,1.097803,1.072162,0.389194,0.142069,-0.154863,-0.049971,-0.164187,0.142069


Using Clements 1930 data on starch accumulated in 12h to estimate starch, hexose and sucrose content when diel fluctuations are ignored.

In [4]:
xlist = list()
ylist = list()
ylist2 = list()
ylist3 = list()
ylist4 = list()
for i in df_matureleaf_content_1.index:
    if i == 4:
        continue
    xlist.append(i)
    ylist.append(min(df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"][i],
                     df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_matureleaf_content_1["sucrose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["sucrose_dusk (g/100gFW)"][i]))
    ylist3.append(min(df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"][i]))
    ylist4.append(min(df_matureleaf_content_1["starch_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["starch_dusk (g/100gFW)"][i]))
df_leaf_content_4 = pd.DataFrame(data={"Series":xlist,
                                       "baseline starch (g/100gFW)":ylist4,
                                       "baseline hemicellulose (g/100gFW)":ylist3,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_leaf_content_4.set_index("Series",drop=True,inplace=True)
df_leaf_content_4

,baseline starch (g/100gFW),baseline hemicellulose (g/100gFW),baseline sucrose (g/100gFW),baseline simple sugars (g/100gFW)
Series,,,,
1,0.424568,1.089463,0.271941,0.032023
2,0.497429,0.663310,0.187004,0.353374
3,0.889538,0.861645,0.097705,0.413878


Extract amino acid content from Chiozza et al 2010, Figure 2

<img src="references/Screenshot_20190717_143619.png" height=50% width=50%>

In [5]:
df_leaf_content_3 = pd.DataFrame(data={"Variety":["aphid resitant","aphid sensitive"],
                                       "GABA (nmol/50mgFW)":[0.8287671232876719,1.0136986301369868],
                                       "Asn (nmol/50mgFW)":[67.26027397260277,86.1643835616437],
                                       "Gln (nmol/50mgFW)":[82.87671232876716,99.31506849315056],
                                       "Glt (nmol/50mgFW)":[184.58904109589022,234.93150684931462],
                                       "His (nmol/50mgFW)":[11.778523489932848,13.657718120805297],
                                       "Pro (nmol/50mgFW)":[19.062499999999943,20.68181818181808],
                                       "Ser (nmol/50mgFW)":[87.28205128205093,100.4102564102559]})
df_leaf_content_3.set_index("Variety",drop=True,inplace=True)
df_leaf_content_3

,GABA (nmol/50mgFW),Asn (nmol/50mgFW),Gln (nmol/50mgFW),Glt (nmol/50mgFW),His (nmol/50mgFW),Pro (nmol/50mgFW),Ser (nmol/50mgFW)
Variety,,,,,,,
aphid resitant,0.828767,67.260274,82.876712,184.589041,11.778523,19.062500,87.282051
aphid sensitive,1.013699,86.164384,99.315068,234.931507,13.657718,20.681818,100.410256


In [6]:
biomassMets = {"sSUCROSE_b":1000*find_average(df_leaf_content_4["baseline sucrose (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight,
               "Starch_b":1000*find_average(df_leaf_content_4["baseline starch (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight,
               "GLC_c":1000*find_average(df_leaf_content_4["baseline simple sugars (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight,
               "ASN_c":100*0.001*find_average(df_leaf_content_3["Asn (nmol/50mgFW)"])/50,
               "sGABA_b":100*0.001*find_average(df_leaf_content_3["GABA (nmol/50mgFW)"])/50,
               "sGLN_b":100*0.001*find_average(df_leaf_content_3["Gln (nmol/50mgFW)"])/50,
               "sGLU_b":100*0.001*find_average(df_leaf_content_3["Glt (nmol/50mgFW)"])/50,
               "HIS_c":100*0.001*find_average(df_leaf_content_3["His (nmol/50mgFW)"])/50,
               "PRO_c":100*0.001*find_average(df_leaf_content_3["Pro (nmol/50mgFW)"])/50,
               "sSER_b":100*0.001*find_average(df_leaf_content_3["Ser (nmol/50mgFW)"])/50
              }


rxn = Reaction("Biomass_leaf_tx")
for met in biomassMets:
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*biomassMets[met]})
rxn.upper_bound = 1000
rxn.lower_bound = 0

leaf_model.add_reaction(rxn)



In [7]:
from sweetlovegroup.transform import setupC3DielModel
leaf_model = setupC3DielModel(leaf_model)

0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
bHIS_v1 --> bHIS_v2
0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2


In [9]:
leaf_model.reactions.Phloem_output_tx1.objective_coefficient = 0
leaf_model.reactions.Phloem_output_tx2.objective_coefficient = 0

leaf_model.reactions.Photon_tx1.upper_bound = 500
leaf_model.reactions.Photon_tx1.lower_bound = 500

from cobra.flux_analysis import pfba
sol = pfba(leaf_model)

In [10]:
io.write_sbml_model(leaf_model,"Data/Soy_leaf_model_GA.sbml")

### Summary

- Since amino acid content was only available in g/gFW and FW/DW data was not available, diel starch and sucrose levels (in g/gFW) were used to predict baseline starch and sucrose levels in leaf; instead of other available data


## Generate stem model

In [11]:
stem_model = model.copy()

Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [12]:
df_stem_content = pd.DataFrame(data={"Series":[1,2,3,4],
                                     "simple sugars_dawn (g/100gFW)":[0.08928305478569998,1.0069230769230781,
                                                                      0.22941176470588265,0.755800478743633],
                                     "sucrose_dawn (g/100gFW)":[0.24999412110523256,0.10918376068376556,
                                                                0.14117647058823524,0.36680114242250283],
                                     "simple sugars_dusk (g/100gFW)":[0.27423868312757205,0.9930341880341897,
                                                                      0.23368616214394122,0.8292719666520014],
                                     "sucrose_dusk (g/100gFW)":[0,0.08606410256411046,
                                                                0.1748626327321765,0.2553567833200341]})
df_stem_content.set_index("Series",drop=True,inplace=True)
df_stem_content["avg. Δsimple sugars (g/100gFW)"] = df_stem_content["simple sugars_dusk (g/100gFW)"]- \
                                                        df_stem_content["simple sugars_dawn (g/100gFW)"]
df_stem_content["avg. Δsucrose (g/100gFW)"] = df_stem_content["sucrose_dusk (g/100gFW)"]- \
                                                        df_stem_content["sucrose_dawn (g/100gFW)"]

xlist = list()
ylist = list()
ylist2 = list()
for i in df_stem_content.index:
    xlist.append(i)
    ylist.append(min(df_stem_content["simple sugars_dawn (g/100gFW)"][i],
                     df_stem_content["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_stem_content["sucrose_dawn (g/100gFW)"][i],
                      df_stem_content["sucrose_dusk (g/100gFW)"][i]))
df_stem_content_std = pd.DataFrame(data={"Series":xlist,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_stem_content_std.set_index("Series",drop=True,inplace=True)
df_stem_content_std

,baseline sucrose (g/100gFW),baseline simple sugars (g/100gFW)
Series,,
1,0.000000,0.089283
2,0.086064,0.993034
3,0.141176,0.229412
4,0.255357,0.755800


### Summary

- Sucrose, hexose content data is per FW and conversion to DW is not available

In [13]:
biomassMets = {"sSUCROSE_b":1000*find_average(df_stem_content_std["baseline sucrose (g/100gFW)"])/ \
                                        stem_model.metabolites.get_by_id("SUCROSE_c").formula_weight,
               "GLC_c":1000*find_average(df_stem_content_std["baseline simple sugars (g/100gFW)"])/ \
                                        stem_model.metabolites.get_by_id("GLC_c").formula_weight
              }

rxn = Reaction("Biomass_stem_tx")
for met in biomassMets:
    rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*biomassMets[met]})
rxn.lower_bound = 0
rxn.upper_bound = 1000

stem_model.add_reaction(rxn)

In [36]:
stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 1
stem_model.reactions.Sucrose_tx.lower_bound = 1
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Phloem_output_tx.objective_coefficient=0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1

from cobra.flux_analysis import pfba
sol = pfba(stem_model)
print(stem_model.reactions.Biomass_stem_tx.flux)

0.5415495157940694


In [14]:
io.write_sbml_model(stem_model,"Data/Soy_stem_model_GA.sbml")

## Generate root model

In [15]:
root_model = model.copy()

#### Gathering root fatty-acid content from Thomas et al 2007 Table I

In [16]:
df_root_content_1 = pd.DataFrame(data={"Tissue type":["Conrad - Epidermis","OX760-6 - Epidermis",
                                                      "Conrad - Endodermis","OX760-6 - Endodermis"],
                                       "Fatty acid 16:1":[0.21,0.05,0.82,0.39],
                                       "Fatty acid 16:0":[0.39,0.25,1.50,0.72],
                                       "Fatty acid 18:1":[0.71,0.18,7.46,1.46],
                                       "Fatty acid 18:0":[0.32,0.04,0.83,0.30],
                                       "Fatty acid 20:0":[0.05,0.03,0.28,0.49],
                                       "Fatty acid 22:0":[0.09,0.04,0.31,0.31],
                                       "Total (ug/cm2)":[1.77,0.59,11.20,3.67]})
df_root_content_1.set_index("Tissue type",inplace=True,drop=True)
df_root_content_1

,Fatty acid 16:1,Fatty acid 16:0,Fatty acid 18:1,Fatty acid 18:0,Fatty acid 20:0,Fatty acid 22:0,Total (ug/cm2)
Tissue type,,,,,,,
Conrad - Epidermis,0.21,0.39,0.71,0.32,0.05,0.09,1.77
OX760-6 - Epidermis,0.05,0.25,0.18,0.04,0.03,0.04,0.59
Conrad - Endodermis,0.82,1.50,7.46,0.83,0.28,0.31,11.20
OX760-6 - Endodermis,0.39,0.72,1.46,0.30,0.49,0.31,3.67


In [17]:
df_root_content_1_std = pd.DataFrame(data={"Fatty acids":["16:0","16:1","18:0","18:1","20:0","22:0"],
                                           "fraction":[find_average(df_root_content_1["Fatty acid 16:0"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"]),
                                                     find_average(df_root_content_1["Fatty acid 16:1"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"]),
                                                     find_average(df_root_content_1["Fatty acid 18:0"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"]),
                                                     find_average(df_root_content_1["Fatty acid 18:1"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"]),
                                                     find_average(df_root_content_1["Fatty acid 20:0"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"]),
                                                     find_average(df_root_content_1["Fatty acid 22:0"])/ \
                                                                find_average(df_root_content_1["Total (ug/cm2)"])]})
df_root_content_1_std.set_index("Fatty acids",drop=True,inplace=True)

In [18]:
rxn = Reaction("Root_fatty_acid_composition")
rxn.add_metabolites({root_model.metabolites.get_by_id("PALMITATE_c"):-1*df_root_content_1_std["fraction"]["16:0"],
                     root_model.metabolites.get_by_id("CPD_9245_p"):-1*df_root_content_1_std["fraction"]["16:1"],
                     root_model.metabolites.get_by_id("STEARIC_ACID_p"):-1*df_root_content_1_std["fraction"]["18:0"],
                     root_model.metabolites.get_by_id("OLEATE_CPD_p"):-1*df_root_content_1_std["fraction"]["18:1"],
                     root_model.metabolites.get_by_id("ARACHIDIC_ACID_p"):-1*df_root_content_1_std["fraction"]["20:0"],
                     root_model.metabolites.get_by_id("DOCOSANOATE_p"):-1*df_root_content_1_std["fraction"]["22:0"],
                     root_model.metabolites.get_by_id("Fatty_Acids_p"):1})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)


In [19]:
generateMetaboliteFormula(rxn)

Fatty_Acids_p
C17.77016831108532H33.23099245502032O2.0000000000000004


In [37]:
#No biomass cannot optimmize

In [20]:
io.write_sbml_model(root_model,"Data/Soy_root_model_GA.sbml")

## Generate seed model

In [83]:
seed_model = model.copy()

#### Gathering seed fatty-acid content from Guleria et al 2008 Table I

In [84]:
df_seed_content = pd.DataFrame(data={"seed type":["large","small"],
                               "16:0 (mg/gDW fatty acds)":[132,122],
                               "18:0 (mg/gDW fatty acds)":[29,38],
                               "18:1 (mg/gDW fatty acds)":[269,245],
                               "18:2 (mg/gDW fatty acds)":[513,518],
                               "18:3 (mg/gDW fatty acds)":[56,76]})
df_seed_content.set_index("seed type",inplace=True,drop=True)
df_seed_content

,16:0 (mg/gDW fatty acds),18:0 (mg/gDW fatty acds),18:1 (mg/gDW fatty acds),18:2 (mg/gDW fatty acds),18:3 (mg/gDW fatty acds)
seed type,,,,,
large,132,29,269,513,56
small,122,38,245,518,76


In [85]:
df_seed_content_std = pd.DataFrame(data={"Fatty acids":["16:0","18:0","18:1","18:2","18:3"],
                               "fraction":[float(find_average(df_seed_content["16:0 (mg/gDW fatty acds)"]))/999,
                                           float(find_average(df_seed_content["18:0 (mg/gDW fatty acds)"]))/999,
                                           float(find_average(df_seed_content["18:1 (mg/gDW fatty acds)"]))/999,
                                           float(find_average(df_seed_content["18:2 (mg/gDW fatty acds)"]))/999,
                                           float(find_average(df_seed_content["18:3 (mg/gDW fatty acds)"]))/999]})
df_seed_content_std.set_index("Fatty acids",inplace=True,drop=True)
df_seed_content_std

,fraction
Fatty acids,
16:0,0.127127
18:0,0.033534
18:1,0.257257
18:2,0.516016
18:3,0.066066


In [86]:
rxn = Reaction("Seed_fatty_acid_composition")
rxn.add_metabolites({seed_model.metabolites.get_by_id("PALMITATE_c"):-1*df_seed_content_std["fraction"]["16:0"],
                     seed_model.metabolites.get_by_id("STEARIC_ACID_p"):-1*df_seed_content_std["fraction"]["18:0"],
                     seed_model.metabolites.get_by_id("OLEATE_CPD_p"):-1*df_seed_content_std["fraction"]["18:1"],
                     seed_model.metabolites.get_by_id("Octadecadienoate_p"):-1*df_seed_content_std["fraction"]["18:2"],
                     seed_model.metabolites.get_by_id("LINOLENIC_ACID_p"):-1*df_seed_content_std["fraction"]["18:3"],
                     seed_model.metabolites.get_by_id("Fatty_Acids_p"):1})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

In [87]:
rxn.reaction

'0.06606606606606606 LINOLENIC_ACID_p + 0.25725725725725723 OLEATE_CPD_p + 0.516016016016016 Octadecadienoate_p + 0.12712712712712712 PALMITATE_c + 0.033533533533533534 STEARIC_ACID_p --> Fatty_Acids_p'

In [88]:
generateMetaboliteFormula(rxn)

Fatty_Acids_p
C17.745745745745744H31.516516516516518O1.9999999999999998


In [89]:
rxn = Reaction("Seed_fatty_acid_ACP_composition")
rxn.add_metabolites({seed_model.metabolites.get_by_id("PALMITATE_c"):-1*df_seed_content_std["fraction"]["16:0"],
                     seed_model.metabolites.get_by_id("STEARIC_ACID_p"):-1*df_seed_content_std["fraction"]["18:0"],
                     seed_model.metabolites.get_by_id("OLEATE_CPD_p"):-1*df_seed_content_std["fraction"]["18:1"],
                     seed_model.metabolites.get_by_id("Octadecadienoate_p"):-1*df_seed_content_std["fraction"]["18:2"],
                     seed_model.metabolites.get_by_id("LINOLENIC_ACID_p"):-1*df_seed_content_std["fraction"]["18:3"],
                     seed_model.metabolites.get_by_id("Fatty_Acids_p"):1})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

#### Gather phospholipids, glycolipids, sterols and free fatty acid content from Guleria et al 2008, Table 1

In [90]:
met = Metabolite("Phospholipids_p",name="Phospholipids",compartment="p")

rxn = Reaction("Phospholipid_composition")
rxn.add_metabolites({seed_model.metabolites.get_by_id("L_PHOSPHATIDATE_p"):-1,met:1})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

print(rxn.reaction)

generateMetaboliteFormula(rxn)

L_PHOSPHATIDATE_p --> Phospholipids_p
Phospholipids_p
C35.0H67.0O8.0P1.0


In [91]:
df_seed_content_1 = pd.DataFrame(data={"seed type":["large","small"],
                               "lipid content (mg/gDW)":[241,224],
                               "phospholipids (mg/gDW lipid)":[57.1,58.7],
                               "glycolipids (mg/gDW lipid)":[10.6,10.3],
                               "sterols (mg/gDW lipid)":[26.8,26.4],
                               "free fatty acids (mg/gDW lipid)":[0.1,0.1]})

df_seed_content_1.set_index("seed type",inplace=True,drop=True)
df_seed_content_1

,lipid content (mg/gDW),phospholipids (mg/gDW lipid),glycolipids (mg/gDW lipid),sterols (mg/gDW lipid),free fatty acids (mg/gDW lipid)
seed type,,,,,
large,241,57.1,10.6,26.8,0.1
small,224,58.7,10.3,26.4,0.1


In [92]:
df_seed_content_1std = pd.DataFrame(data={"lipid entities":["free fatty acids","glycolipids","phospholipids","sterols"],
                               "fraction":[float(find_average(df_seed_content_1["free fatty acids (mg/gDW lipid)"]))/find_average(df_seed_content_1["lipid content (mg/gDW)"]),
                                           float(find_average(df_seed_content_1["glycolipids (mg/gDW lipid)"]))/find_average(df_seed_content_1["lipid content (mg/gDW)"]),
                                           float(find_average(df_seed_content_1["phospholipids (mg/gDW lipid)"]))/find_average(df_seed_content_1["lipid content (mg/gDW)"]),
                                           float(find_average(df_seed_content_1["sterols (mg/gDW lipid)"]))/find_average(df_seed_content_1["lipid content (mg/gDW)"])
                                          ]})
df_seed_content_1std.set_index("lipid entities",inplace=True,drop=True)
df_seed_content_1std

,fraction
lipid entities,
free fatty acids,0.000430
glycolipids,0.044946
phospholipids,0.249032
sterols,0.114409


In [93]:
met = Metabolite("Lipids_p",name="Lipids",compartment="p")

rxn = Reaction("Seed_lipid_composition")
rxn.add_metabolites({seed_model.metabolites.get_by_id("Fatty_Acids_p"):-1*df_seed_content_1std["fraction"]["free fatty acids"]/ \
                                                                seed_model.metabolites.get_by_id("Fatty_Acids_p").formula_weight,
                     seed_model.metabolites.get_by_id("Phospholipids_p"):-1*df_seed_content_1std["fraction"]["phospholipids"]/ \
                                                                seed_model.metabolites.get_by_id("Phospholipids_p").formula_weight,
                     met:(df_seed_content_1std["fraction"]["free fatty acids"]/ \
                          seed_model.metabolites.get_by_id("Fatty_Acids_p").formula_weight) + \
                         (df_seed_content_1std["fraction"]["phospholipids"]/ \
                          seed_model.metabolites.get_by_id("Phospholipids_p").formula_weight)
                    })
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)


print(rxn.reaction)

1.5532709068320986e-06 Fatty_Acids_p + 0.0003849771413172512 Phospholipids_p --> 0.00038653041222408327 Lipids_p


cobra/core/metabolite.py:104 UserWarning: 17.745745745745744 is not an integer (in formula C17.745745745745744H31.516516516516518O1.9999999999999998)
cobra/core/metabolite.py:104 UserWarning: 31.516516516516518 is not an integer (in formula C17.745745745745744H31.516516516516518O1.9999999999999998)
cobra/core/metabolite.py:104 UserWarning: 1.9999999999999998 is not an integer (in formula C17.745745745745744H31.516516516516518O1.9999999999999998)


In [98]:
0.0003849771413172512/0.00038653041222408327

0.9959815040221683

In [99]:
leaf_model.reactions.GLUCOKIN_RXN_c1.genes

frozenset({<Gene Glyma.05G110500 at 0x7f5fbde1ac50>,
           <Gene Glyma.17G156200 at 0x7f5fbde1aeb8>})

In [94]:
generateMetaboliteFormula(rxn)

Lipids_p
C34.9306638486788H66.85740976434217O7.975889024133011P0.9959815040221683


#### Generating Biomass equation for seed  
Gather non-reducing sugars, reducing sugars, starch, protein and lipid content from Guleria et al 2008, Table 1

In [95]:
df_seed_content_3 = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "total soluble sugars (mg/gDW)":[125,101],
                               "proteins (mg/gDW)":[377,264],
                               "lipid content (mg/gDW)":[241,224]})

df_seed_content_3.set_index("seed type",inplace=True,drop=True)
df_seed_content_3

,starch (mg/gDW),reducing sugars (mg/gDW),total soluble sugars (mg/gDW),proteins (mg/gDW),lipid content (mg/gDW)
seed type,,,,,
large,49.7,24.9,125,377,241
small,42.3,27.7,101,264,224


In [96]:
rxn = Reaction("Biomass_seed_tx")
rxn.add_metabolites({seed_model.metabolites.get_by_id("Starch_b"):-1*find_average(df_seed_content_3["starch (mg/gDW)"])/ \
                                                                 seed_model.metabolites.get_by_id("Starch_b").formula_weight,
                     seed_model.metabolites.get_by_id("sSUCROSE_b"):-1*find_average(df_seed_content_3["reducing sugars (mg/gDW)"])/ \
                                                                 seed_model.metabolites.get_by_id("sSUCROSE_b").formula_weight,
                     seed_model.metabolites.get_by_id("GLC_c"):-1*(find_average(df_seed_content_3["total soluble sugars (mg/gDW)"])-find_average(df_seed_content_3["reducing sugars (mg/gDW)"]))/ \
                                                                 seed_model.metabolites.get_by_id("GLC_c").formula_weight,
                     seed_model.metabolites.get_by_id("Protein_b"):-1*find_average(df_seed_content_3["proteins (mg/gDW)"])/ \
                                                                 seed_model.metabolites.get_by_id("Protein_b").formula_weight,
                     seed_model.metabolites.get_by_id("Lipids_p"):-1*find_average(df_seed_content_3["lipid content (mg/gDW)"])/ \
                                                                 seed_model.metabolites.get_by_id("Lipids_p").formula_weight
                    })
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

print(rxn.reaction)

0.4812499042495865 GLC_c + 0.3602480102722086 Lipids_p + 0.8038744537410865 Protein_b + 0.2837043898937095 Starch_b + 0.07683397737540275 sSUCROSE_b --> 


cobra/core/metabolite.py:104 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 34.9306638486788 is not an integer (in formula C34.9306638486788H66.85740976434217O7.975889024133011P0.9959815040221683)
cobra/core/metabolite.py:104 UserWarning: 66.85740976434217 is no

In [53]:
backup = seed_model.copy()

In [82]:
seed_model = backup.copy()

seed_model.metabolites.Fatty_acyl_ACP_p.remove_from_model()
seed_model.metabolites.ACP_p.remove_from_model()

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 1
seed_model.reactions.Sucrose_tx.lower_bound = 1
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Phloem_output_tx.objective_coefficient=0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1

from cobra.flux_analysis import pfba
sol = pfba(seed_model)
print(seed_model.reactions.Biomass_seed_tx.flux)

0.0


In [79]:
for rxn in seed_model.metabolites.get_by_id("MALONYL_ACP_p").reactions:
    print(rxn.id+"\t"+str(rxn.reaction))

RXN_9531_p	Decanoyl_ACPs_p + MALONYL_ACP_p + PROTON_p --> 3_oxo_dodecanoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p
RXN_9535_p	Dodecanoyl_ACPs_p + MALONYL_ACP_p + PROTON_p --> 3_oxo_myristoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p
RXN_9527_p	MALONYL_ACP_p + Octanoyl_ACPs_p + PROTON_p --> 3_oxo_decanoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p
RXN_9523_p	Hexanoyl_ACPs_p + MALONYL_ACP_p + PROTON_p --> 3_Oxo_octanoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p
2_PERIOD_3_PERIOD_1_PERIOD_180_RXN_p	ACETYL_COA_p + MALONYL_ACP_p + PROTON_p --> Acetoacetyl_ACPs_p + CARBON_DIOXIDE_p + CO_A_p
RXN_9516_p	Butanoyl_ACPs_p + MALONYL_ACP_p + PROTON_p --> 3_oxo_hexanoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p
MALONYL_COA_ACP_TRANSACYL_RXN_p	ACP_p + MALONYL_COA_p --> CO_A_p + MALONYL_ACP_p
RXN_9539_p	MALONYL_ACP_p + Myristoyl_ACPs_p + PROTON_p --> 3_oxo_palmitoyl_ACPs_p + ACP_p + CARBON_DIOXIDE_p


In [33]:
io.write_sbml_model(seed_model,"Data/Soy_seed_model_GA.sbml")